In [2]:
import requests
from bs4 import BeautifulSoup
import pandas as pd 
import sqlite3
import numpy as np 
from datetime import datetime 

url = 'https://web.archive.org/web/20230902185326/https://en.wikipedia.org/wiki/List_of_countries_by_GDP_%28nominal%29'
table_attribs = ["Country", "GDP_USD_millions"]
db_name = 'World_Economies.db'
table_name = 'Countries_by_GDP'
csv_path = './Countries_by_GDP.csv'

def extract(url, table_attribs):

    html = requests.get(url).text    
    data = BeautifulSoup(html, 'html.parser')
    df = pd.DataFrame(columns = table_attribs)
    
    tables = data.find_all('tbody')

    rows = tables[2].find_all('tr')

    for row in rows:
        col = row.find_all('td')
        if (len(col) != 0):
            if (col[0].find('a') is not None) and (col[2].contents[0] != '—'):
                df_dict = {"Country":col[0].a.contents[0], "GDP_USD_millions":col[2].contents[0]}
                new_df = pd.DataFrame(df_dict, index=[0])
                df = pd.concat([df, new_df], ignore_index = True)

    return df

def transform(df):

    list_of_values = []
    for values in df['GDP_USD_millions']:
        list_of_values.append(values)

    cleaned_values = []
    for values in list_of_values:
        parts = values.split(',')
        cleaned_text = ''.join(parts)
        cleaned_val = np.round((float(cleaned_text) / 1000), 2)
        cleaned_values.append(cleaned_val)
    df["GDP_USD_millions"] = cleaned_values

    df = df.rename(columns = {'GDP_USD_millions':'GDP_USD_billions'})
    return df

def load_to_csv(df, csv_path):
    ''' This function saves the final dataframe as a `CSV` file 
    in the provided path. Function returns nothing.'''
    df.to_csv(csv_path)

def load_to_db(df, sql_connection, table_name):
    df.to_sql(table_name, sql_connection, if_exists='replace', index=False)

def run_query(query_statement, sql_connection):
    print(query_statement)
    query_output = pd.read_sql(query_statement, sql_connection)
    print(query_output)

def log_progress(message):
    timestamp_format = '%Y-%h-%d-%H:%M:%S'
    now = datetime.now()
    timestamp = now.strftime(timestamp_format)
    with open("./etl_project_log.txt", "a") as f:
        f.write(timestamp + " : " + message + '\n')

log_progress("Preliminaries complete. Initiating ETL process.")

df = extract(url, table_attribs)
log_progress("Data extraction complete. Initiating Transformation process.")

df = transform(df)
log_progress("Data transformation complete. Initiating loading process.")

load_to_csv(df, csv_path)
log_progress("Data saved to CSV file.")

conn = sqlite3.connect(db_name)
log_progress("SQL Connection initiated.")

load_to_db(df, conn, table_name)
log_progress("Data loaded to Database as table. Running the query.")

statement = f"SELECT * from {table_name} WHERE GDP_USD_billions >= 100"
run_query(statement, conn)
log_progress("Process Complete.")
conn.close()


SELECT * from Countries_by_GDP WHERE GDP_USD_billions >= 100
          Country  GDP_USD_billions
0   United States          26854.60
1           China          19373.59
2           Japan           4409.74
3         Germany           4308.85
4           India           3736.88
..            ...               ...
64          Kenya            118.13
65         Angola            117.88
66           Oman            104.90
67      Guatemala            102.31
68       Bulgaria            100.64

[69 rows x 2 columns]
